# **Lab 4: Approximation**
**Dániel Szabó**

# **Abstract**

This laboratory is about approximating a function $f\in V$ by calculating its projection $P_Nf\in V_N\subset V$, where $V_N$ is a finite dimensional subspace of $V$. The $L^2$ projection $P_Nf$ of function $f\in L^2([a,b])$ onto $V_N\subset L^2([a,b])$ is defined by $\forall v\in V_N\ (P_Nf,v)=(f,v)$ with the inner product $(f,g)=\int_a^bf(x)g(x)dx$.

#**About the code**

In [20]:
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2021 Dániel Szabó (dszabo@kth.se)

# This file is part of the course DD2365 Methods in Scientific Computing
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This template is maintained by Johan Hoffman
# Please report problems to jhoffman@kth.se

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

In [21]:
# Load neccessary modules.
import numpy as np
import scipy.integrate as integrate
import random

random.seed(0)

# **Introduction**

With a basis $\{\phi_i\}_{i=1}^N$ of $V_N$ the task of calculating the projection of a function $f$ onto $V_N$ can be formulated as a matrix equation $A\alpha=b$ where $a_{ij}=(\phi_i,\phi_j)$ and $b_i=(f,\phi_i)$. From the solution vector $\alpha$ we can calculate the projection: $P_N f(x)=\sum_{i=1}^N\alpha_i\phi_i(x)$. If we use the Lagrange basis, it is also true that $\alpha_i=f(x_i)$, where $x_i$ are the nodes of the given mesh.

# **Method**

Method "mesh_check" checks if the input array is a valid 1D mesh, i.e. that its elements are increasing.

Method "lagrange" calculates the value of the first order Lagrange polynomial described by the input parameters. $\lambda_{k,0}(x)=\frac{x_{k+1}-x}{x_{k+1}-x_k}$ and $\lambda_{k,1}(x)=\frac{x-x_k}{x_{k+1}-x_k}$.

Method "assemble" follows Algorithm 9.2 of the lecture notes. Using the fact that $\phi_k(x)=\begin{cases}\lambda_{k,1}(x) \textrm{ if } x\in[x_{k-1},x_{k}]\\\lambda_{{k+1},0}(x) \textrm{ if } x\in[x_{k},x_{k+1}]\end{cases}$ we can rewrite the matrix and vector value updates (where we only look at one interval at a time) with the Lagrange polynomials. For each interval $[x_k,x_{k+1}]$, values $\int_{x_k}^{x_{k+1}}f(x)\lambda_{k,i}(x)dx$ are stored in the local $b$ vector; and values $\int_{x_k}^{x_{k+1}}\lambda_{k,i}(x)\lambda_{k,j}(x)dx$ are stored in the local $A$ matrix (for $i,j\in\{0,1\}$). Then the corresponding parts of the global matrix and vector are updated with the local ones.

Method "projection" checks if the input mesh is a valid mesh. Then it calculates matrix $A$ and vector $b$ using the previous method, and returns vector $\alpha$ that satisfies $A\alpha=b$.

In [22]:
def mesh_check(mesh):
    for i in range(len(mesh)-1):
        if mesh[i] >= mesh[i+1]:
            raise Exception("Not a valid mesh")

def lagrange(x,k,t,mesh):
    if t==0:
        return (mesh[k+1]-x)/(mesh[k+1]-mesh[k])
    if t==1:
        return (x-mesh[k])/(mesh[k+1]-mesh[k])

def assemble(f, mesh):
    q_plus_1 = 1+1 # q=1, but q+1 is used more often, so this value is stored
    n_o_intervals = len(mesh)-1
    A = np.zeros((n_o_intervals+1, n_o_intervals+1)) # global A matrix
    B = np.zeros(n_o_intervals+1) # global b vector
    for k in range(n_o_intervals):
        a = np.zeros((q_plus_1,q_plus_1)) # local A matrix
        b = np.zeros(q_plus_1) # local b vector
        for i in range(q_plus_1):
            b[i] = integrate.quad(lambda x: f(x)*lagrange(x,k,i,mesh), mesh[k], mesh[k+1])[0]
            for j in range(q_plus_1):
                a[i,j] = integrate.quad(lambda x: lagrange(x,k,i,mesh)*lagrange(x,k,j,mesh), mesh[k], mesh[k+1])[0]
        A[k:k+q_plus_1,k:k+q_plus_1] += a
        B[k:k+q_plus_1] += b
    return A, B

def projection(f, mesh):
    mesh_check(mesh)
    A,b = assemble(f,mesh)
    return np.linalg.solve(A,b)

# **Results**

The verification of the method is done by computing the results for a random function: a polynomial that has degree between 2 and 10, and its coefficients are drawn from the standard normal distribution. Method "f_ex" returns the value of this polynomial at a given point. (Of course it is possible to return the values of any function -- not only polynomials -- by calling "f_ex" if we modify its code, like for example if we use the commented line instead of the rest.) Method "print_poly" prints a polynomial given by its array of coefficients in a readable way.



In [23]:
degree = random.randint(2,10)
coefficients = []
for i in range(degree+1):
    coefficients.append(random.gauss(0,1))

def f_ex(x):
    # return np.sin(x**2)
    res = 0
    for i in range(degree+1):
        res += coefficients[i] * x**i
    return res

def print_poly(coeffs):
    print(coeffs[0], end='')
    for i in range(1,len(coeffs)):
        print(" + ",coeffs[i]," * x^",i, end='', sep='')
    print("")

print("f(x) = ",end='',sep='')
print_poly(coefficients)

f(x) = 0.05219198828260849 + -1.0434089742005737 * x^1 + -0.06700651572905797 * x^2 + 1.1947466787499987 * x^3 + -1.4471231264273656 * x^4 + 0.9843395512466481 * x^5 + -0.37407684698672905 * x^6 + 1.0746503857790555 * x^7 + -1.8919635634885046 * x^8


For the testing of our method, we generate a mesh with subinterval length $h$. First, $h=1/2$, then it is halved again and again until it becomes small enough. For each value of $h$, the result $v$ of our method is calculated, as well as the exact result $v^*$, i.e. the values of the function at the points of the mesh. We calculate the error $||v-v^*||$, and check whether it is smaller than the error of the previous iteration, so if reducing the subinterval length makes the result converge to the correct solution. We also check if the final error is close to zero. Invalid input handling is also tested.

In [24]:
previous_error=np.inf
h = 1/2
while h > 2.5 * 10**-4:
    mesh = np.arange(0,1+h,h)
    approx = projection(f_ex, mesh)
    correct = [f_ex(x_i) for x_i in mesh]
    error = np.linalg.norm(np.subtract(approx, correct))
    #print(error)
    assert(error < previous_error)
    previous_error = error
    h /= 2
np.testing.assert_almost_equal(error, 0, decimal=4)

# Invalid mesh test
np.testing.assert_raises(Exception, projection, f_ex, [0,0.5,0.5,1])

# **Discussion**

The method, implemented for solving the tasks, actually succeeded in solving it, as it is confirmed by the test results.

If the threshold for $h$ in the testing part is chosen to be much smaller than the current value (0.00025), then the computation takes significantly more time. This is due to the fact that while $h$ is decreasing exponentially, the number of iterations in our method is growing exponetially.